Criação do Alerta

In [4]:
import requests
import pandas as pd
from plyer import notification
from datetime import datetime
import sqlite3

In [7]:
def extracao(url,base):
    data = str(datetime.now())
    req = requests.get(url)
    status_code = req.status_code
    data_json = req.json()
    if status_code == 200:
        df = pd.DataFrame(data_json)
        message = 'Requisição realizada com sucesso'
        notification.notify(
        title=f'ATENÇÃO: {message}'
        , message = message
        , app_name='alerta'
        , timeout=10)

        return(df)
        
    elif status_code >= 400:
        message = 'Erro na requisição'

    else:
        print ('Outros tipos de erro')
    
    notification.notify(
        title=f'ATENÇÃO: {message}'
        , message = str('Carregamento da base %s. \n %s' % (base,data))
        , app_name=''
        , timeout=10)

Extração das bases

In [8]:
base_municipios = extracao('https://brasilapi.com.br/api/ibge/municipios/v1/sp?providers=dados-abertos-br,gov,wikipedia','base_municipios')

In [9]:
base_municipios.head()

,nome,codigo_ibge
0,ADAMANTINA,3500105
1,ADOLFO,3500204
2,AGUAI,3500303
3,AGUAS DA PRATA,3500402
4,AGUAS DE LINDOIA,3500501


In [10]:
base_uf = extracao('https://brasilapi.com.br/api/ibge/uf/v1','base_uf')

In [11]:
base_uf.head()

,id,sigla,nome,regiao
0,11,RO,Rondônia,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
1,12,AC,Acre,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,13,AM,Amazonas,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
3,14,RR,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
4,15,PA,Pará,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"


In [12]:
base_estado_sp = extracao('https://brasilapi.com.br/api/ibge/uf/v1/sp','base_estado_sp')

In [13]:
base_estado_sp.head()

,id,sigla,nome,regiao
id,35,SP,São Paulo,3
nome,35,SP,São Paulo,Sudeste
sigla,35,SP,São Paulo,SE


In [14]:
base_pix = extracao('https://brasilapi.com.br/api/pix/v1/participants','base_pix')

In [15]:
base_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03T09:30:00.000Z
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03T09:30:00.000Z
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03T09:30:00.000Z


Tratamento das bases

In [16]:
base_uf.head()

,id,sigla,nome,regiao
0,11,RO,Rondônia,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
1,12,AC,Acre,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,13,AM,Amazonas,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
3,14,RR,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
4,15,PA,Pará,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"


In [17]:
sigla_regiao = [sr['sigla'] for sr in base_uf['regiao']]
nome_regiao = [nr['nome'] for nr in base_uf['regiao']]
sigla_estado = [estado for estado in base_uf['sigla']]
nome_estado = [estado for estado in base_uf['nome']]

In [18]:
base_uf_tratada = pd.DataFrame({'Sigla': sigla_estado, 'Nome': nome_estado, 'Sigla Região': sigla_regiao, 'Nome Região': nome_regiao})

In [19]:
base_uf_tratada.head()

,Sigla,Nome,Sigla Região,Nome Região
0,RO,Rondônia,N,Norte
1,AC,Acre,N,Norte
2,AM,Amazonas,N,Norte
3,RR,Roraima,N,Norte
4,PA,Pará,N,Norte


In [41]:
type(base_uf_tratada)

pandas.core.frame.DataFrame

In [20]:
base_estado_sp.head()

,id,sigla,nome,regiao
id,35,SP,São Paulo,3
nome,35,SP,São Paulo,Sudeste
sigla,35,SP,São Paulo,SE


In [21]:
base_estado_sp_tratada = base_estado_sp.reset_index()

In [22]:
base_estado_sp_tratada

,index,id,sigla,nome,regiao
0,id,35,SP,São Paulo,3
1,nome,35,SP,São Paulo,Sudeste
2,sigla,35,SP,São Paulo,SE


In [23]:
base_estado_sp_tratada1 = base_estado_sp_tratada[base_estado_sp_tratada['index'] == 'nome']

In [24]:
base_estado_sp_tratada1

,index,id,sigla,nome,regiao
1,nome,35,SP,São Paulo,Sudeste


In [25]:
base_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03T09:30:00.000Z
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03T09:30:00.000Z
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03T09:30:00.000Z


In [26]:
from datetime import datetime

In [27]:
base_pix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ispb                     797 non-null    object
 1   nome                     797 non-null    object
 2   nome_reduzido            797 non-null    object
 3   modalidade_participacao  797 non-null    object
 4   tipo_participacao        797 non-null    object
 5   inicio_operacao          797 non-null    object
dtypes: object(6)
memory usage: 37.5+ KB


In [28]:
base_pix['inicio_operacao'] = pd.to_datetime(base_pix['inicio_operacao'])

In [29]:
base_pix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   ispb                     797 non-null    object             
 1   nome                     797 non-null    object             
 2   nome_reduzido            797 non-null    object             
 3   modalidade_participacao  797 non-null    object             
 4   tipo_participacao        797 non-null    object             
 5   inicio_operacao          797 non-null    datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 37.5+ KB


In [30]:
base_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03 09:30:00+00:00
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03 09:30:00+00:00
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03 09:30:00+00:00
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03 09:30:00+00:00
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03 09:30:00+00:00


In [31]:
base_pix['inicio_operacao'] = base_pix['inicio_operacao'].dt.strftime("%d/%m/%Y")

In [32]:
base_pix.head()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,03/11/2020
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,03/11/2020
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,03/11/2020
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,03/11/2020
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,03/11/2020


Salvar em banco de dados

In [47]:
def salva_bd(df, nome_tabela):
    '''
        Salva dataframe df na tabela nome_tabela.
    '''
    conn = sqlite3.connect('banco_projeto.db')

    # Escrever o DataFrame na tabela 'nome_tabela'
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

    conn.close()

    return True

In [49]:
salva_bd(base_uf_tratada, 'base_uf')

True

In [50]:
salva_bd(base_estado_sp_tratada1, 'base_estado_sp')

True

In [51]:
salva_bd(base_municipios, 'base_municipios')

True

In [52]:
salva_bd(base_pix, 'base_pix')

True

In [54]:
def tabelas_bd():
    '''
        Retorna um dataframe com as tabelas do banco de dados.
    '''
    conn = sqlite3.connect('banco_projeto.db')

    # Executar uma consulta que retorna as informações do esquema do banco de dados
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()

    return schema

In [55]:
tabelas_bd()

,name
0,base_uf_tratada
1,base_uf
2,base_estado_sp
3,base_municipios
4,base_pix
